In [ ]:
import json, requests, pandas, time, re
from datetime import datetime

#### 2 PARAMETROS OBLIGATORIOS: MAIL/USERID Y CAMPOS DESEADOS

In [ ]:
# se puede llamar al servicio con emails o tracker ids, en forma de lista
emails = ["patriciatojo@hotmail.com", "veronica.seminario@despegar.com", "jcorigliano@despegar.com"]
users = ['d7c4c073-ef9a-493f-84c0-73ef9a393f0a', '644ef9f3-204d-4a25-8ef9-f3204d3a25ae']

# Campos seleccionados
campos = ["userid", "email", "datetime", "pr", "fl", "ci", "co", "dc", "pri"]

#### PARAMETROS OPCIONALES:

In [ ]:
from_date = "15/05/2016"
start_ts = int(time.mktime(datetime.strptime(from_date, "%d/%m/%Y").timetuple())) * 1000

to_date = "10/07/2016"
end_ts = int(time.mktime(datetime.strptime(to_date, "%d/%m/%Y").timetuple())) * 1000

last_n_obs = "100"

flows_select = "SEARCH,DETAIL,CHECKOUT,THANKS"

prods_select = "FLIGHTS,HOTELS"

brands_select = "ALL_DESPEGAR" 

countries_select = "AR,BR"

#### Funcion que llama al servicio y descarga datos del json

In [ ]:
def getUserHistory(email_or_trackerid):
    #WAIT_SECONDS = 0.2
    #time.sleep(WAIT_SECONDS)
    
    userid_format = re.compile('^[a-z0-9]{8}(\\-[a-z0-9]{4}){3}\\-[a-z0-9]{12}$')
    kind = "user" if re.search(userid_format, email_or_trackerid) else "email"
    amount_param = "amount=" + last_n_obs if last_n_obs else "amount=" + 1000 # por default pido 1000
    flow_param = "&flows=" + flows_select if flows_select else ""
    prod_param = "&products=" + prods_select if prods_select else ""
    country_param = "&countries=" + countries_select if flows_select else ""
    brand_param = "&brands=" + brands_select if flows_select else ""
    to_ts_param = "&to_ts=" + str(end_ts) if end_ts else ""
    from_ts_param = "&from_ts=" + str(start_ts) if start_ts else ""

    url = "http://10.2.7.6/euler-service/v3/" + kind + "/" + email_or_trackerid + "/history?" + flow_param + prod_param + country_param + brand_param + from_ts_param + to_ts_param
    headers = {'X-Client': 'Vero Analytics', 'X-Cluster': 'euler-web-beta-h2'}
    rfm_get = requests.get(url, headers=headers)
    return rfm_get.json()

# funcion que selecciona solo los campos pedidos y los devuelve como key-value    
def selectFields(event, columns):
    return {k: event[k] for k in event if str(k) in columns}

#for k in event.iterkeys():
#        if str(k) in columns:
#            return {k: event[k]}

#### Aplico la función a un RDD de una lista de userids o emails

In [ ]:
usersRDD = sc.parallelize(emails)

In [ ]:
histories = usersRDD.flatMap(getUserHistory)

In [ ]:
histories.first()

In [ ]:
# en el json, los campos estan adentro de la llave de actionData. 
# Primero hago flatmap para aplanar el json
histories = usersRDD.flatMap(getUserHistory)    .map(lambda event: event['actionData'])    .map(lambda event: selectFields(event, campos))

In [ ]:
# El problema para pasar de rdd a dataframe con createDataFrame es que inferschema infiere el schema del primer dict, 
# pero no todos los dict tienen el mismo schema (porque no todos los eventos tienen los mismos campos)
sqlContext.createDataFrame(histories).printSchema()

In [ ]:
# Hago yo el schema, asumiendo que son todos strings
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType,LongType

def inferirSchema(campos):
    schema_campo = []
    for k in campos:
        if k in ['hc', 'hr', 'rev', 'utype', 'hc', 'geoid', 'pg', 'days', 'dt', 'nr', 'rn', 'pssgrcount']:
            schema_campo.append(StructField(k,  IntegerType(), True))
        elif k in ['datetime', 'timestamp', 'hid', 'idcro']:
            schema_campo.append(StructField(k, LongType(), False))
        else:
            schema_campo.append(StructField(k, StringType(), True))
    return StructType(schema_campo)

schema = inferirSchema(campos)

print schema

In [ ]:
# create dataframe
df = sqlContext.createDataFrame(histories, schema)

In [ ]:
# create Pandas dataframe

historiesPanda = df.toPandas()

historiesPanda

### EXTRAS

In [ ]:
### funcion que llama al servicio y descarga datos del json
def getUserHistory(userid):
    WAIT_SECONDS = 0.2
    time.sleep(WAIT_SECONDS)
    url = "http://10.2.7.6/euler-service/v3/user/" + userid + "/history/"
    headers = {'X-Client': 'Vero Analytics', 'X-Cluster': 'euler-web-beta-h2'}
    rfm_get = requests.get(url, headers=headers)
    return rfm_get.json()
    
# funcion que selecciona solo los campos pedidos y los devuelve como key-value    
def selectFields(event, columns):
    return {k: event[k] for k in event if str(k) in columns}

In [ ]:
### como funciona match para saber el formato de userid?
userid_format = re.compile('^[a-z0-9]{8}(\\-[a-z0-9]{4}){3}\\-[a-z0-9]{12}$')

# si hay match, el objeto existe, si no no.
match = re.search(userid_format, 'd7c4c073-ef9a-493f-84c0-73ef9a393f0a')
if match:
    print 'ok'

In [ ]:
### asi hubiera hecho el schema si todos los structype fueran string ¬¬
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField(k,  StringType(), True) for k in campos])

In [ ]:
# es lo mismo que hacer campo por campo
#schema = StructType([StructField('userId', StringType(), True),
#                     StructField('timestamp', StringType(), True),
#                     StructField('pr', StringType(), True),
#                     StructField('fl', StringType(), True),
#                     StructField('ci', StringType(), True),
#                     StructField('co', StringType(), True),
#                     StructField('dc', StringType(), True),
#                     StructField('pri', StringType(), True)])